# Tutorial: Quick Translation of GTFS to GTFS-PLUS

In [ ]:
import os,datetime,shutil
import pandas as pd

# Download GTFS

In [ ]:
GTFS_LINK  = r"http://admin.gotransitnc.org/sites/default/files/developergtfs/GoRaleigh_GTFS_0.zip"

BASE_DIR   = os.getcwd()
NEW_FOLDER = "GoRaleigh_GTFS"
GTFS_LOC   = os.path.join(BASE_DIR,NEW_FOLDER)

In [ ]:
# Download the file from the URL and unzip
from urllib import urlopen
from zipfile import ZipFile

try:
    os.stat(os.path.join(BASE_DIR,NEW_FOLDER))
except:
    os.mkdir(os.path.join(BASE_DIR,NEW_FOLDER)) 


tempzip_filename = os.path.join(BASE_DIR,NEW_FOLDER,"tempgtfs.zip")
zipresp = urlopen(GTFS_LINK)
tempzip = open(tempzip_filename, "wb")
tempzip.write(zipresp.read())
tempzip.close()
zf = ZipFile(tempzip_filename)
zf.extractall(path = os.path.join(BASE_DIR,NEW_FOLDER))
zf.close()
os.remove(tempzip_filename)

## Validate GTFS Feed
Make sure you are starting with a valid network.
This can take a while for a large network. 

In [ ]:
import transitfeed
loader        = transitfeed.Loader(GTFS_LOC, memory_db=True)
schedule      = loader.Load()
schedule.Validate()

In [ ]:
print "Routes Loaded:"
rts = [r.route_long_name for r in schedule.routes.itervalues()]
for r in rts:
    print " - ",r

## Add needed data to turn GTFS to GTFS-PLUS
There are files that we need to add:  
  * routes_ft.txt  
  * vehicles_ft.txt  
  * trips_ft.txt  
  * transfers_ft.txt 
  * walk_access_ft.txt 

In [ ]:
import csv
import gtfs_plus

In [ ]:
GTFS_PLUS_LOC   = "GoRaleigh_GTFS_PLUS"
OUTPUT_DIR      = os.path.join(BASE_DIR,GTFS_PLUS_LOC)

# start with the GTFS files if you don't have these already
try:
    shutil.copytree(GTFS_LOC, "GoRaleigh_GTFS_PLUS")
    # copy over the config file from the earlier tutorials
    shutil.copy(os.path.join(BASE_DIR,"tta","input","demand-single","config_ft.txt"), 
                os.path.join(OUTPUT_DIR, "config_ft.txt"))
except:
    # hopefully this is ok and you're just doing this multiple times
    pass

DEFAULT_MODE    = "local_bus"
DEFAULT_VEHICLE = "standard_bus"
SEATED_CAPACITY   = 30
STANDING_CAPACITY = 20
MAX_SPEED         = 45
ACCELERATION      = 3
DECELERATION      = 4
DWELL = r'"3 + 2*[boards] + 1.5*[alights]"'

### Create `routes_ft.txt` 
For now, assume a default mode

In [ ]:
route_modes_dict = gtfs_plus.routesft_assume_mode(schedule, DEFAULT_MODE)
with open(os.path.join(OUTPUT_DIR,'routes_ft.txt'),'wb') as f:
    f.write("route_id,mode\n")
    w = csv.writer(f)
    w.writerows(route_modes_dict.items())

### Create `trips_ft.txt` 
For now, assume a default vehicle

In [ ]:
trip_vehicle_dict = dict(zip(schedule.trips.keys(),[DEFAULT_VEHICLE]*len(schedule.trips.keys())))
with open(os.path.join(OUTPUT_DIR,'trips_ft.txt'),'wb') as f:
    f.write("trip_id,vehicle_name\n")
    w = csv.writer(f)
    w.writerows(trip_vehicle_dict.items())

### Create `vehicles_ft.txt` 
FOr now, assume mostly defaults

In [ ]:
with open(os.path.join(OUTPUT_DIR,'vehicles_ft.txt'),'wb') as f:
    f.write("vehicle_name,seated_capacity,standing_capacity,max_speed,acceleration,deceleration,dwell_formula\n")
    f.write("%s,%d,%d,%4.2f,%4.2f,%4.2f,%s\n"%(DEFAULT_VEHICLE,SEATED_CAPACITY,STANDING_CAPACITY,MAX_SPEED,ACCELERATION,DECELERATION,DWELL))

### Create `transfers_ft.txt` 


In [ ]:
xfer_dict = gtfs_plus.create_tranfers(schedule,max_xfer_dist=0.6)

with open(os.path.join(OUTPUT_DIR,'transfers_ft.txt'),'wb') as f:
    f.write("from_stop_id,to_stop_id,dist\n")
    for k,v in xfer_dict.iteritems():
        f.write("%s,%s,%4.2f\n" % (k[0],k[1],v))
        #and reverse link
        f.write("%s,%s,%4.2f\n" % (k[1],k[0],v))

### `walk_access.txt` 
Should be created based on the geography of the demand you are using.